In [1]:
import sys
import os
sys.path.append("/Users/baeyeongmin/Desktop/workspace/dacon-2024-gbt-hackerton")

In [2]:
from src import root_path
from src.data_processing import load_data
train_df, test_df = load_data(root_path=root_path)

In [3]:
import pygwalker as pyg

In [4]:
walker = pyg.walk(train_df[train_df['분류'] != '지역'])

Box(children=(HTML(value='<div id="ifr-pyg-0" style="height: auto">\n    <head>\n        <meta http-equiv="Con…

In [3]:
import re
def normalize_title(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text.strip()

def normalize_keywords(text):
    text = re.sub(r'[^가-힣\s,]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.strip()

def clean_keywords(keywords):
    keywords = keywords.split(',')
    keywords = [keyword.strip() for keyword in keywords if keyword.strip()]
    return ' '.join(keywords)

train_df['title'] = train_df['제목'].apply(normalize_title)
train_df['keywords'] = train_df['키워드'].apply(lambda x: clean_keywords(normalize_keywords(x)))
train_df['text'] = train_df['title'] + ' [SEP] ' + train_df['keywords']

test_df['title'] = test_df['제목'].apply(normalize_title)
test_df['keywords'] = test_df['키워드'].apply(lambda x: clean_keywords(normalize_keywords(x)))
test_df['text'] = test_df['title'] + ' [SEP] ' + test_df['keywords']

In [4]:
def get_length(text):
    return len(text.split())

# 제목, 키워드, 전체 텍스트의 길이 계산
train_df['title_length'] = train_df['title'].apply(get_length)
train_df['keywords_length'] = train_df['keywords'].apply(get_length)
train_df['total_length'] = train_df['text'].apply(get_length)

# 길이 통계 출력
print("제목 길이 통계:")
print(train_df['title_length'].describe())
print("\n키워드 길이 통계:")
print(train_df['keywords_length'].describe())
print("\n전체 텍스트 길이 통계:")
print(train_df['total_length'].describe())

제목 길이 통계:
count    54609.000000
mean         7.443645
std          2.077995
min          1.000000
25%          6.000000
50%          7.000000
75%          9.000000
max         27.000000
Name: title_length, dtype: float64

키워드 길이 통계:
count    54609.000000
mean       165.425424
std        113.492817
min          4.000000
25%         99.000000
50%        141.000000
75%        203.000000
max       2628.000000
Name: keywords_length, dtype: float64

전체 텍스트 길이 통계:
count    54609.000000
mean       173.869069
std        113.964916
min          6.000000
25%        107.000000
50%        150.000000
75%        212.000000
max       2637.000000
Name: total_length, dtype: float64


In [5]:
from src.data_processing import get_label_encoded

train_df, label_encoder = get_label_encoded(train_df)

## Class imbalance problem
### Training Stage
1. model 1: 지역=1, 지역이 아닌 라벨=0 으로 한 binanry classification model
2. model 2: 라벨이 0으로 classification model에 대해서 다시 나머지 55개 주제에 대해서 classification model(지역 라벨에 대한 로우들은 모두 제거한 데이터로 학습+Focal loss)

### inference Stage
- model 1 추론 후 model 2 추론하면 끝.


In [8]:
from src.data_processing import get_label_encoded

# 기존 레이블 인코딩 유지
train_df, label_encoder = get_label_encoded(train_df)

# 모델 1을 위한 이진 분류 레이블 생성
train_df['label_model1'] = (train_df['분류'] == '지역').astype(int)

# 모델 2를 위한 데이터셋 생성 (지역 제외)
train_df_model2 = train_df[train_df['분류'] != '지역'].copy()
train_df_model2['label_model2'] = train_df_model2['label']

# 결과 확인
print("모델 1 데이터:")
print(train_df[['분류', 'label', 'label_model1']].head(10))
print("\n모델 1 레이블 분포:")
print(train_df['label_model1'].value_counts())

print("\n모델 2 데이터:")
print(train_df_model2[['분류', 'label', 'label_model2']].head(10))
print("\n모델 2 레이블 분포:")
print(train_df_model2['label_model2'].value_counts())

# 데이터셋 크기 확인
print("\n전체 데이터셋 크기:", len(train_df))
print("모델 2 데이터셋 크기:", len(train_df_model2))

모델 1 데이터:
          분류  label  label_model1
0   문화:전시_공연      0             0
1         지역      1             1
2         지역      1             1
3         지역      1             1
4         국제      2             0
5   문화:전시_공연      0             0
6      정치:선거      3             0
7  경제:금융_재테크      4             0
8         지역      1             1
9         지역      1             1

모델 1 레이블 분포:
label_model1
0    27659
1    26950
Name: count, dtype: int64

모델 2 데이터:
           분류  label  label_model2
0    문화:전시_공연      0             0
4          국제      2             2
5    문화:전시_공연      0             0
6       정치:선거      3             3
7   경제:금융_재테크      4             4
11   사회:의료_건강      5             5
12   정치:행정_자치      6             6
14   정치:국회_정당      7             7
17     스포츠:축구      8             8
18  경제:서비스_쇼핑      9             9

모델 2 레이블 분포:
label_model2
10    3454
21    2568
33    2318
18    1480
11     995
7      966
5      950
20     845
22     841
25     711
0      6

In [9]:
train_df_model2


,ID,분류,제목,키워드,title,keywords,text,title_length,keywords_length,total_length,label,label_model1,label_model2
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화...","용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최",용인문화재단 인문학 콘서트 뮤지컬 이야기 개최 인문학 콘서트 뮤지컬 이야기 용인문화...,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최 [SEP] 용인...",8,88,97,0,0,0
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코...","용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할",용인시 스페인 미국 국제 명예 자문관 위촉 역할 대외 홍보 지원 용인시 권태면 주코...,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할 [SEP] 용인시 스페...",8,111,120,2,0,2
5,TRAIN_00005,문화:전시_공연,"소강석 목사 ""한미 참전용사 끝까지 찾아뵐 것""","소강석,목사,한미,참전용사,한미,참전,용사,초청,행사,방한,초청,종료,한미,참전,용...","소강석 목사 ""한미 참전용사 끝까지 찾아뵐 것""",소강석 목사 한미 참전용사 한미 참전 용사 초청 행사 방한 초청 종료 한미 참전 용...,"소강석 목사 ""한미 참전용사 끝까지 찾아뵐 것"" [SEP] 소강석 목사 한미 참전용...",7,105,113,0,0,0
6,TRAIN_00006,정치:선거,"[지방선거 D-360] 전 현직 공직출신 후보군, 행정경험 무기로 '단체장' 노린다","현직,공직,출신,후보군,행정,경험,무기,단체장,지방,선거,1년,현직,공직,출신,인사...","[지방선거 D-360] 전 현직 공직출신 후보군, 행정경험 무기로 '단체장' 노린다",현직 공직 출신 후보군 행정 경험 무기 단체장 지방 선거 년 현직 공직 출신 인사 ...,"[지방선거 D-360] 전 현직 공직출신 후보군, 행정경험 무기로 '단체장' 노린다...",10,298,309,3,0,3
7,TRAIN_00007,경제:금융_재테크,"용인시, '자동차세 연납제도'로 9.15% 세액공제","용인시,자동차세,연납,제도,9.15%,세액,공제,용인시청,용인시,최대,10%,자동차...","용인시, '자동차세 연납제도'로 9.15% 세액공제",용인시 자동차세 연납 제도 세액 공제 용인시청 용인시 최대 자동차세 세액 공제 각종...,"용인시, '자동차세 연납제도'로 9.15% 세액공제 [SEP] 용인시 자동차세 연납...",5,116,122,4,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54598,TRAIN_54598,사회:사건_사고,"“귀신 들렸다 개똥 핥아라” 물고문 숨진 10살 여아, 이모는 무속인","귀신,개똥,물고문,여아,이모,무속인,살인,아동학대,혐의,이모,이모부,기소,화장실,손...","“귀신 들렸다 개똥 핥아라” 물고문 숨진 10살 여아, 이모는 무속인",귀신 개똥 물고문 여아 이모 무속인 살인 아동학대 혐의 이모 이모부 기소 화장실 손...,"“귀신 들렸다 개똥 핥아라” 물고문 숨진 10살 여아, 이모는 무속인 [SEP] 귀...",10,185,196,21,0,21
54600,TRAIN_54600,문화:종교,[중부포토] 동지(冬至) 맞아 팥죽 준비해요.,"동지,冬至,팥죽,준비,절기,동지,冬至,하루,대각사,용인시,처인구,스님,신도들,팥죽,새알",[중부포토] 동지(冬至) 맞아 팥죽 준비해요.,동지 팥죽 준비 절기 동지 하루 대각사 용인시 처인구 스님 신도들 팥죽 새알,[중부포토] 동지(冬至) 맞아 팥죽 준비해요. [SEP] 동지 팥죽 준비 절기 동지...,5,13,19,35,0,35
54601,TRAIN_54601,사회:사회일반,"생물자원관, '러브버그' 대발생 대응법 모색 전문가 ""독성 전파 질병 없어""","생물자원관,러브버그,모색,대응법,전문가,독성,전파,질병,양가희,시민,불편,초래,러브...","생물자원관, '러브버그' 대발생 대응법 모색 전문가 ""독성 전파 질병 없어""",생물자원관 러브버그 모색 대응법 전문가 독성 전파 질병 양가희 시민 불편 초래 러브...,"생물자원관, '러브버그' 대발생 대응법 모색 전문가 ""독성 전파 질병 없어"" [SE...",10,126,137,18,0,18
54604,TRAIN_54604,국제,"용인 아파트서 30대 여성, 아들 딸 함께 추락 ""극단 선택 추정""","용인,아파트,여성,아들,추락,극단,선택,추정,경찰,현장,유서,경위,조사,처인구,경기...","용인 아파트서 30대 여성, 아들 딸 함께 추락 ""극단 선택 추정""",용인 아파트 여성 아들 추락 극단 선택 추정 경찰 현장 유서 경위 조사 처인구 경기...,"용인 아파트서 30대 여성, 아들 딸 함께 추락 ""극단 선택 추정"" [SEP] 용인...",11,55,67,2,0,2
